In [23]:
import os
from pprint import pprint
from proxy import Resolve

r = Resolve('/Volumes/DIT-2T#3/220826-虾米音乐节@阿那亚/素材/220826-Day_01')

r.project.GetSetting()
r.project.SetSetting('isAutoColorManage', '0')

True